In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Imputer
from sklearn.impute import KNNImputer

# Helper Functions
from function import date_time_convertion, list_to_cols

In [2]:
DIR = "KuaiRec 2.0/"

rootpath="./"
print(rootpath)

./


In [3]:
# print("Loading big matrix...")
# big_matrix = pd.read_csv(rootpath + "data/big_matrix.csv")
print("Loading small matrix...")
small_matrix = pd.read_csv(rootpath + "data/small_matrix.csv")

print("Loading social network...")
social_network = pd.read_csv(rootpath + "data/social_network.csv")
social_network["friend_list"] = social_network["friend_list"].map(eval)

print("Loading item features...")
item_categories = pd.read_csv(rootpath + "data/item_categories.csv")
item_categories["feat"] = item_categories["feat"].map(eval)

print("Loading user features...")
user_features = pd.read_csv("data/user_features.csv")

print("Loading items' daily features...")
item_daily_features = pd.read_csv("data/item_daily_features.csv")

print("All data loaded.")

Loading small matrix...
Loading social network...
Loading item features...
Loading user features...
Loading items' daily features...
All data loaded.


## Data Exploration

Since we are mainly work with small_matrix our explorations are going to focus on that dataset.

In [4]:
small_matrix.head()

,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,14,148,4381,6067,2020-07-05 05:27:48.378,20200705.0,1.593898e+09,0.722103
1,14,183,11635,6100,2020-07-05 05:28:00.057,20200705.0,1.593898e+09,1.907377
2,14,3649,22422,10867,2020-07-05 05:29:09.479,20200705.0,1.593898e+09,2.063311
3,14,5262,4479,7908,2020-07-05 05:30:43.285,20200705.0,1.593898e+09,0.566388
4,14,8234,4602,11000,2020-07-05 05:35:43.459,20200705.0,1.593899e+09,0.418364


In [5]:
# sum([True for idx, row in small_matrix.iterrows() if any(row.isnull())])
print(f"Small Matrix shape: {small_matrix.shape}")
rows_wnans = small_matrix.shape[0] - small_matrix.dropna().shape[0]
print(f"There are {rows_wnans} rows with NaNs")

Small Matrix shape: (4676570, 8)
There are 181992 rows with NaNs


Since there are 181991 rows that have NaNs we will impute them in order not to lose valuable data. We could the compare how our results differe if we were to only drop these rows or impute them.

## Data Preparation

In [6]:
import pandas as pd
import numpy as np
import datetime as dt

def date_time_convertion(col):
    col = pd.to_datetime(col)
    min_col = col.min()
    for idx, value in enumerate(col):
        new_val = (value - min_col).total_seconds()
        col.iloc[idx] = new_val
    return col

small_matrix["time"] = date_time_convertion(small_matrix["time"])

/var/folders/0f/xqjk7pr56mjd12xvy1w435_40000gn/T/ipykernel_6018/1452114024.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '97462.318' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  col.iloc[idx] = new_val


In [7]:
# merged_df = pd.merge(small_matrix, item_categories, on="video_id", how="left")
# print(merged_df.shape)

# # merged_df = pd.merge(small_matrix, item_categories, on="video_id", how="inner")
# # print(merged_df.shape) 
# # This to ensure that every video_id in small_matrix is also in item_categories

# merged_df = pd.merge(merged_df, social_network, on="user_id", how="left")
# print(merged_df.shape)

# # merged_df = pd.merge(small_matrix, social_network, on="user_id", how="inner")
# # print(merged_df.shape) 


In [11]:
small_matrix_merged = (
    small_matrix
    .merge(item_categories, on="video_id", how="left")
    # .merge(social_network, on="user_id", how="left")
)
display(small_matrix_merged.head())

,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio,feat
0,14,148,4381,6067,97462.318,20200705.0,1.593898e+09,0.722103,"[11, 28, 19]"
1,14,183,11635,6100,97473.997,20200705.0,1.593898e+09,1.907377,[28]
2,14,3649,22422,10867,97543.419,20200705.0,1.593898e+09,2.063311,[9]
3,14,5262,4479,7908,97637.225,20200705.0,1.593898e+09,0.566388,[25]
4,14,8234,4602,11000,97937.399,20200705.0,1.593899e+09,0.418364,[6]


In [12]:
small_matrix_merged = list_to_cols(small_matrix_merged, "feat")

In [13]:
# Data Imputation

# KNN Imputation
# imputer = KNNImputer(n_neighbors=20)
# df_filled = imputer.fit_transform(small_matrix)

# Median/Mean/Mode Imputation
for col in small_matrix_merged.columns:
    small_matrix_merged[col] = small_matrix_merged[col].fillna(small_matrix_merged[col].median()) # median() or mode()[0]

/var/folders/0f/xqjk7pr56mjd12xvy1w435_40000gn/T/ipykernel_6018/226922022.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  small_matrix_merged[col] = small_matrix_merged[col].fillna(small_matrix_merged[col].median()) # median() or mode()[0]
